<a href="https://colab.research.google.com/github/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [2]:
import datasets
#raw_datasets = load_dataset("imdb")

## Importing the dataset


In [2]:
df = pd.read_json("D:\pan20-authorship-verification-training-small\pan20-authorship-verification-training-small.jsonl", lines = True)
df_labels = pd.read_json("D:\pan20-authorship-verification-training-small\pan20-authorship-verification-training-small-truth.jsonl", lines = True)

In [17]:
text = df['pair'][0][0]
text = text[0:13000]

In [15]:
def text_segmentate(text, maxlen, seps='\n', strips=None):
    """将文本按照标点符号划分为若干个短句
    """
    text = text.strip().strip(strips)
    if seps and len(text) > maxlen:
        pieces = text.split(seps[0])
        text, texts = '', []
        for i, p in enumerate(pieces):
            if text and p and len(text) + len(p) > maxlen - 1:
                texts.extend(text_segmentate(text, maxlen, seps[1:], strips))
                text = ''
            if i + 1 == len(pieces):
                text = text + p
            else:
                text = text + p + seps[0]
        if text:
            texts.extend(text_segmentate(text, maxlen, seps[1:], strips))
        return texts
    else:
        return [text]

In [18]:
datas = []

text1 = text_segmentate(text, maxlen=510, seps='.?!')
text2 = text_segmentate(df['pair'][23][1], maxlen=510, seps='.?!')

print(len(text1), len(text2))

while len(text1) < 30 or len(text2) < 30:
    print("HOHOHO")
    if len(text1) < 30:
        n_text1 = []
        for i in range(30):
            for sent in text1:
                n_text1.append(sent)
        text1 = n_text1
    elif len(text2) < 30:
        n_text2 = []
        for i in range(30):
            for sent in text2:
                n_text2.append(sent)
        text2 = n_text2
datas.append((text1, text2))

print(len(text1), len(text2))

28 46
HOHOHO
840 46


In [23]:
len(datas[0][0][3])

487

In [3]:
df.tail()
#df['pair'][0][0]

,id,fandoms,pair
52596,3926a1bd-6d33-5513-a694-12d7590443d4,"[True Blood, Austin & Ally]",[Enjoy! And time stands still beneath the air ...
52597,17227d87-a3cb-5aca-b5fd-bb642e88b030,"[True Blood, Kuroko no Basuke/黒子のバスケ]","[""I forgive you,"" she blurted out looking stra..."
52598,0332dcaa-7230-508b-badb-820f9d8eebaa,"[Austin & Ally, Five Nights at Freddy´s]","[""What? Why? What""s happening? Where are we go..."
52599,f5e4884a-13fd-51fb-a880-053d6655d59a,"[Five Nights at Freddy´s, Austin & Ally]","[""Gah!"" I growled. ""I need a password to acces..."
52600,900c4a19-e22b-59fd-9cc7-8490919b1696,"[Five Nights at Freddy´s, Austin & Ally]",[Chica heard a noise below her and saw Golden ...


We take first 2,000 and last 2,000 sentences from the dataset to ensure uniform distribution of labels

In [52]:
batch_1 = pd.concat([df[:2000], df[-2000:]])
#batch_1.reset_index(drop=True, inplace=True)
batch_1_labels = pd.concat([df_labels[:2000], df_labels[-2000:]])
#batch_1_labels.reset_index(drop=True, inplace=True)
batch_1_labels['labels'] = batch_1_labels['same'].astype(int) 
batch_1 = batch_1.join(batch_1_labels['labels'])
batch_1 = batch_1.drop(['fandoms', 'id'], axis=1)
#batch_1.index = np.arange(0, len(batch_1))
batch_1.reset_index(drop=True, inplace=True)


In [4]:
#load full set without selecting 4000
df_labels['labels'] = df_labels['same'].astype(int) 
batch_1 = df.join(df_labels['labels'])
batch_1 = batch_1.drop(['fandoms', 'id'], axis=1)


In [7]:
dataset = datasets.Dataset.from_pandas(batch_1)

In [8]:
tokenizer = ppb.AutoTokenizer.from_pretrained("bert-base-cased")

In [9]:
def tokenize_function(examples):
    return tokenizer(examples['pair'][0], examples['pair'][1], truncation=True, padding=True, max_length=255)

In [10]:
tokenized_dataset = dataset.map(tokenize_function)

52601ex [1:09:44, 12.57ex/s]


In [ ]:
tokenized_dataset.save_to_disk(r"C:\Users\ivank\Documents\BERT_projects")

In [12]:
tokenized_dataset = tokenized_dataset.remove_columns(['pair'])

In [15]:
tokenized_dataset['labels'][:20]

[0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1]

In [14]:
tokenized_dataset = tokenized_dataset.shuffle(seed=42)

In [16]:
tokenized_dataset_notest = tokenized_dataset.select(range(47340))
tokenized_dataset_fortest = tokenized_dataset.select(range(47340, 52601))

In [17]:
tokenized_dataset_tosplit = tokenized_dataset_notest.train_test_split(test_size=0.1)

In [18]:
small_train_dataset = tokenized_dataset_tosplit['train']
small_eval_dataset = tokenized_dataset_tosplit['test']

In [19]:
model = ppb.AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels = 2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [20]:
training_args = ppb.TrainingArguments('test_trainer', evaluation_strategy="epoch")#, label_names=["labels"])

In [ ]:
from datasets import list_metrics

list_metrics()

In [125]:
trainer = ppb.Trainer(model=model, args=training_args, 
train_dataset=small_train_dataset, eval_dataset=small_eval_dataset,
compute_metrics=compute_metrics)

In [127]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
100%|██████████| 125/125 [00:04<00:00, 25.56it/s]


{'eval_loss': 0.693314254283905,
 'eval_accuracy': 0.507,
 'eval_runtime': 5.1022,
 'eval_samples_per_second': 195.995,
 'eval_steps_per_second': 24.499,
 'epoch': 3.0}

In [126]:
trainer.train()

***** Running training *****
  Num examples = 9000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3375
 15%|█▍        | 500/3375 [01:12<06:43,  7.12it/s]Saving model checkpoint to test_trainer\checkpoint-500
Configuration saved in test_trainer\checkpoint-500\config.json


{'loss': 0.697, 'learning_rate': 4.259259259259259e-05, 'epoch': 0.44}


Model weights saved in test_trainer\checkpoint-500\pytorch_model.bin
 30%|██▉       | 1000/3375 [02:24<05:32,  7.14it/s]Saving model checkpoint to test_trainer\checkpoint-1000
Configuration saved in test_trainer\checkpoint-1000\config.json


{'loss': 0.6943, 'learning_rate': 3.518518518518519e-05, 'epoch': 0.89}


Model weights saved in test_trainer\checkpoint-1000\pytorch_model.bin
 33%|███▎      | 1125/3375 [02:50<05:16,  7.10it/s]  ***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
                                                   
 33%|███▎      | 1126/3375 [02:55<1:00:31,  1.61s/it]

...

Model weights saved in test_trainer\checkpoint-2500\pytorch_model.bin
 89%|████████▉ | 3000/3375 [07:32<00:53,  6.97it/s]Saving model checkpoint to test_trainer\checkpoint-3000
Configuration saved in test_trainer\checkpoint-3000\config.json


{'loss': 0.7003, 'learning_rate': 5.555555555555556e-06, 'epoch': 2.67}


Model weights saved in test_trainer\checkpoint-3000\pytorch_model.bin
100%|██████████| 3375/3375 [08:28<00:00,  7.18it/s]***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
                                                   
100%|██████████| 3375/3375 [08:33<00:00,  7.18it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 3375/3375 [08:33<00:00,  6.57it/s]

{'eval_loss': 0.693314254283905, 'eval_accuracy': 0.507, 'eval_runtime': 5.0181, 'eval_samples_per_second': 199.277, 'eval_steps_per_second': 24.91, 'epoch': 3.0}
{'train_runtime': 513.3337, 'train_samples_per_second': 52.597, 'train_steps_per_second': 6.575, 'train_loss': 0.6994329517505787, 'epoch': 3.0}


TrainOutput(global_step=3375, training_loss=0.6994329517505787, metrics={'train_runtime': 513.3337, 'train_samples_per_second': 52.597, 'train_steps_per_second': 6.575, 'train_loss': 0.6994329517505787, 'epoch': 3.0})

In [108]:
from datasets import load_metric

metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions = predictions, references = labels)

In [21]:
#del model
#del trainer
#del pytorch_model
torch.cuda.empty_cache()

In [22]:
small_train_dataset.set_format('torch')
small_eval_dataset.set_format('torch')

In [23]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=10)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=10)


In [26]:
model = ppb.AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels = 2)
model.to(device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [27]:
optimizer = ppb.AdamW(model.parameters(), lr=2e-5)
num_epochs=5
num_training_steps=num_epochs*len(train_dataloader)
lr_scheduler = ppb.get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)


In [28]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

100%|██████████| 21305/21305 [1:01:55<00:00,  6.21it/s]

In [30]:
from datasets import load_metric

metric = load_metric("f1")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'f1': 0.7979550102249489}

In [4]:
"""
for p, pair in enumerate(batch_1['pair']):
    #for i, text in enumerate(pair):
    #    pair[i] = text[0:250]
    batch_1['pair'][p] = ' [SEP] '.join(pair)
"""
# code for splitting before tokenization

We check how many pairs are labeled as `same` (value 1) and how many are labeled `different` (having the value 0)

In [5]:
batch_1_labels['same'].value_counts()

1    2000
0    2000
Name: same, dtype: int64

## Loading the Pre-trained BERT model

In [25]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = "cpu"
print(device)

cuda:0


In [1]:
# DistilBERT:
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# BERT:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

# Avoid unnecessary warnings
ppb.logging.set_verbosity_error()

NameError: name 'ppb' is not defined

The variable `model` holds a pretrained BERT model.

## Model #1: Preprocessing and Tokenization

Tokenization is performed using BertTokenizer and may take 7-10 minutes

In [63]:
tokenized = batch_1['pair'].apply((lambda x: tokenizer(text = x[0], text_pair = x[1],
add_special_tokens=True, truncation=True, max_length=201, padding=True, return_tensors="pt").to(device)))

In [61]:
tokenized[0] #let's check what the tokenized input looks like

{'input_ids': tensor([[  101,  1045,  5670,  1037,  2978,  1010, 23625,  5599,  2026,  2159,
         14957,  2013,  2028, 13547,  2000,  1996,  2060,  1011,  1011,  2021,
          2026,  2159,  2024,  4738,  2006,  1996,  8659, 13547,  1996,  2087,
          1012,  2066,  7570,  9890,  1012,  1012,  1012,  2061,  2066,  7570,
          9890,  1012,  1012,  1012,  2002,  4332,  1037,  2978,  1010,  1998,
          2256,  2159,  3113,  3495,  1012,  1045,  2064,  1000,  1056,  6235,
          2009,  1012,  1012,  1012,  1999,  2023,  2279,  2617,  1010,  1045,
          2123,  1000,  1056,  2298,  2185,  1010,  2129,  9596,  2009,  3849,
          1012,  1045,  6237,  2046,  2010,  2159,  1012,  2027,  1000,  2128,
          2066,  7570,  9890,  1000,  1055,  1012,  1012,  1012,  2027,  2024,
           102,  1000,  2035,  2097,  2468,  2028,  2007,  3607,  1010,  1000,
          2002,  2056,  1010,  2471,  3432,  1010,  2010, 15138, 18823,  1012,
         10006,  2020,  2525,  8555,  

In [172]:
#tokenized.index = np.arange(0, len(tokenized))
#tokenized = tokenized.reset_index(drop=True, inplace=True)

#re-indexing was required for some experiments, but not amymore. Use re-indexing when you iterate over enumerated rows. 

#tokenized_index[1999]
#tokenized_index[2000]

In [129]:
"""
for i, pair in enumerate(tokenized):
    #input_id = tokenized[pair]['input_ids']
    #tokenized[i] = ' [SEP] '.join(pair['input_ids'])
    #if i == 0:
    #    print(i, "PPAAAAIR", pair)
    #    print("000", tokenized[i]['input_ids'])
    #print(i)
    texts = tokenized[i]['input_ids'][0] + tokenized[i]['input_ids'][1]
    types = tokenized[i]['token_type_ids'][0] + tokenized[i]['token_type_ids'][1]
    masks = tokenized[i]['attention_mask'][0] + tokenized[i]['attention_mask'][1]
    if i > 1999:
        #print(text)
        tokenized[i]['input_ids'] = texts
        tokenized[i]['token_type_ids'] = types
        tokenized[i]['attention_mask'] = masks
"""        

# code for splitting after tokenization. Not used in batched processing.\

"""
token_tensors = []
for i, pair in enumerate(tokenized):
    token_tensors.append(pair['input_ids'])

token_tensors_df = pd.DataFrame(token_tensors)
"""

# code for extracting the encodings. Not used in batched processing.

In [ ]:
"""
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
"""
# code for padding when it is not performed during tokenization

# to simulate padding:
# padded = np.array([i for i in token_tensors_df.values])

#np.array(padded).shape

In [ ]:
"""
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape
"""

# adding attention mask if it is not performed during tokenization

## Model #1 Learning

The `model()` function runs our sentences through BERT (it is inherited by BERT's `forward()` function). The results of the processing will be returned into `all_output`

In [10]:
#print(torch.cuda.memory_summary(device=None, abbreviated=False))
torch.cuda.empty_cache()

In [ ]:
"""
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

print(device)
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
model = model.to(device)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
"""

# standard training with separate, manually created encodings and attention masks (no batching => may require a lot of memory)

In [220]:
# test model to try different modes 

test_input = tokenizer(["Hello, my dog is cute", "And what now?"], padding=True, return_tensors="pt").to("cuda:0")
type(test_input)
model = model.to("cuda:0")
test_output = model(**test_input)

In [11]:
model = model.to(device)


all_outputs = torch.Tensor()
all_outputs = all_outputs.to(device)

with torch.no_grad():
    for batch in tokenized:
        
        #batch['input_ids'].to(device)              # not needed if CUDA was enabled during tokenization
        #batch['token_type_ids'].to(device)
        #batch['attention_mask'].to(device)

        outputs = model(**batch)
        all_outputs = torch.cat((all_outputs, outputs.last_hidden_state), 0)
    print(all_outputs.shape)       

torch.Size([4000, 40, 768])


In [64]:
tokenized.index = np.arange(0, len(tokenized))
# tokenized.reset_index()#drop=True, inplace=True)

In [76]:
tokenized[0]

KeyError: 'Indexing with integers (to access backend Encoding for a given batch index) is not available when using Python based tokenizers'

In [77]:
from transformers import Trainer

In [ ]:
trainer = 

In [72]:
#dataset = torch.utils.data.TensorDataset(tokenized, torch.Tensor(labels))

In [31]:
#model = model.to(device)

#with torch.no_grad():
#    features = tokenized.apply(lambda x: model(**x).last_hidden_state)


Extract `[CLS]` tokens to used them as input to simple LogReg classifier and save them in the `features` variable

In [39]:
features = all_outputs[:,0,:].cpu().numpy() # [:,0,:] meaning: ':' for all sequences, '0' for first token in sequence, ':' for all 768 hidden layers

The labels indicating which pair is written by the same author or by different ones go into the `labels` variable

We also randomly permute labels and then permute features according to the same pattern. This is done to provide uniform distribution of lables after train/test split

In [40]:
labels = batch_1_labels['same']

idx = np.random.permutation(labels.index)
labels = labels.reindex(idx)
features = pd.DataFrame(features)
features = features.reindex(idx)

In [60]:
dataset = TensorDataset(torch.Tensor(features), torch.Tensor(labels))


NameError: name 'TensorDataset' is not defined

## Model #2: Train/Test Split
We now split the obtained datset into a training set and testing set in standard proportion (75% to 25%)

In [52]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [45]:
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scores: ', grid_search.best_score_)

best parameters:  {'C': 100.0}
best scores:  0.6076666666666667


In [53]:
lr_clf = LogisticRegression(C=97)
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=97)

In [54]:
lr_clf.score(test_features, test_labels)

0.603

In [55]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.503 (+/- 0.00)
